# Goal: simulate market dataset such that we can correctly estimate coefficients later on 



In [1]:
import import_ipynb
import BLP_functions as f 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from numpy.linalg import inv
import builtins

np.set_printoptions(precision=3, suppress=True)
np.set_printoptions(legacy='1.13')

n_market = 50
n_product = 5
n_consumer = 100
market_variance = 0.01 #amount of variation between markets 
endogenous_noise = 0.01 #how noise to generate price 

#structural parameters 
true_alpha = 3 #decrease with utility 
true_beta = 13 #increase with utility
true_sigma_alpha = 3.5
true_sigma_beta = 7
market_variance = 0.01 #amount of variation between markets 

In [2]:
all_markets = []
for market in range(n_market):
    x = np.random.uniform(0, 1, n_product)
    instrument = np.random.uniform(0, 10, n_product)

    # generate market shares and price variables endogenously 
    price = f.price_gen(x, instrument, endogenous_noise)
    inner_utility = f.utility_gen([true_alpha, true_beta], [true_sigma_alpha, true_sigma_beta], x, price)
    inner_market_utility = f.market_gen(inner_utility, market_variance)
    inner_prob_choice = f.choice_prob(pd.DataFrame(inner_market_utility))
    # generate market share directly from the choice probabilities
    outer_market_share = inner_prob_choice.sum(axis=0)/n_consumer

    market_j = pd.DataFrame({
        'market_id': market,
        'product_id': list(range(n_product)),
        'x': x, 
        'instrument': instrument, 
        'price': price,
        'share': outer_market_share, 
        'share_not_buy': 1 - sum(outer_market_share)
    })
    # print(market)
    all_markets.append(market_j)

print(all_markets[:50])

export_dataset = pd.concat(all_markets, ignore_index=True)
export_dataset.to_csv('simulated_market_data.csv', index=False)

[   market_id  product_id         x  instrument     price     share  \
0          0           0  0.493082    2.518164  1.719542  0.266252   
1          0           1  0.346661    3.629889  2.309982  0.015682   
2          0           2  0.615159    5.721881  3.679873  0.162275   
3          0           3  0.497563    4.259378  2.754934  0.042154   
4          0           4  0.210252    9.112006  5.534184  0.092999   

   share_not_buy  
0       0.420638  
1       0.420638  
2       0.420638  
3       0.420638  
4       0.420638  ,    market_id  product_id         x  instrument     price     share  \
0          1           0  0.287193    0.992381  0.713134  0.151744   
1          1           1  0.456526    5.036393  3.236159  0.005105   
2          1           2  0.499921    9.302838  5.840275  0.086931   
3          1           3  0.819922    4.197053  2.883301  0.485024   
4          1           4  0.259223    1.437189  0.977818  0.037687   

   share_not_buy  
0       0.233508  
1   